In [8]:
example = """
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...
""".strip().splitlines()

with open("day20.txt", "r") as f:
    data = f.readlines()

In [94]:
from typing import List, Iterator, Iterable, Tuple

class ImageTile(object):
    def __init__(self, id: int, pixels: Iterable[str]):
        self.id = id
        self.pixels = tuple(pixels)

        self.patterns = {
            self.top, # Top
            self.right, # Right (rotate -90)
            self.bottom, # Bottom (rotate 180)
            self.left, # Left (rotate 90)
            "".join(reversed(self.top)), # Top
            "".join(reversed(self.right)), # Right (rotate -90)
            "".join(reversed(self.bottom)), # Bottom (rotate 180)
            "".join(reversed(self.left)), # Left (rotate 90)
        }

    @property
    def top(self) -> str:
        return self.pixels[0]
        
    @property
    def bottom(self) -> str:
        return self.pixels[9]
        
    @property
    def left(self) -> str:
        return "".join(self.pixels[i][0] for i in range(10))
        
    @property
    def right(self) -> str:
        return "".join(self.pixels[i][9] for i in range(10))

    def get_pattern_side(self, pattern: str) -> str:
        sides = ["top", "left", "right", "bottom"]
        for side in sides:
            if pattern in [getattr(self, side), "".join(reversed(getattr(self, side)))]:
                return side
        
        return None

    def flip(self, axis: str):
        if axis == "y":
            self.pixels = tuple(reversed(self.pixels))
        elif axis == "x":
            self.pixels = tuple(
                "".join(reversed(row))
                for row in self.pixels
            )
        else:
            raise NotImplementedError(f"Cannot flip about the '{axis}' (?) axis.")

    def transpose(self):
        self.pixels = tuple(
            "".join(self.pixels[y][x] for y in range(10))
            for x in range(10)
        )

    def move(self, side: str, to: str):
        if side == to:
            return

        ops = {
            "top-left": lambda: [self.transpose()],
            "top-right": lambda: [self.transpose(), self.flip("x")],
            "top-bottom": lambda: [self.flip("y")],

            "left-top": lambda: [self.transpose()],
            "left-right": lambda: [self.flip("x")],
            "left-bottom": lambda: [self.transpose(), self.flip("y")],

            "right-top": lambda: [self.flip("x"), self.transpose()],
            "right-left": lambda: [self.flip("x")],
            "right-bottom": lambda: [self.transpose()],


            "bottom-top": lambda: [self.flip("y")],
            "bottom-left": lambda: [self.flip("y"), self.transpose()],
            "bottom-right": lambda: [self.transpose()]
        }

        ops[f"{side}-{to}"]()

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return isinstance(other, ImageTile) and self.id == other.id

    def __repr__(self):
        return f"Tile({self.id})"

    def __str__(self):
        return f"Tile({self.id})"

    def borderless(self) -> Tuple[str]:
        return tuple(
            row[1:-1]
            for row in self.pixels[1:-1]
        )

    @staticmethod
    def from_text(text: List[str]) -> Iterator["ImageTile"]:
        tile_id = None
        tile_pixels = []
        for line in text:
            line = line.strip()
            if line == "":
                if tile_id is not None:
                    yield ImageTile(int(tile_id), tile_pixels)

                tile_id = None
                tile_pixels = []

            elif "Tile " in line:
                tile_id = line[5:-1]
            else:
                tile_pixels.append(line)

        if tile_id is not None:
            yield ImageTile(int(tile_id), tile_pixels)

example_tiles = list(ImageTile.from_text(example))
assert len(example_tiles) == 9

for tile in example_tiles:
    assert 0 < tile.id < 10000
    assert len(tile.pixels) == 10

assert example_tiles[0].borderless() == tuple([
    "#..#....",
    "...##..#",
    "###.#...",
    "#.##.###",
    "#...#.##",
    "#.#.#..#",
    ".#....#.",
    "##...#.#",
])

assert example_tiles[0].get_pattern_side("..##.#..#.") == "top"
assert example_tiles[0].get_pattern_side("###..###..") == "bottom"
assert example_tiles[0].get_pattern_side(".#####..#.") == "left"
assert example_tiles[0].get_pattern_side("...#.##..#") == "right"

real_tiles = list(ImageTile.from_text(data))
assert len(real_tiles) == 144


In [164]:
from typing import Dict, Tuple, Set
from collections import defaultdict, Counter
from math import sqrt

class Image(object):
    def __init__(self, tiles: Iterator[ImageTile]):
        self.tiles = list(tiles)
        self.height = self.width = int(sqrt(len(self.tiles)))

        lookup = defaultdict(lambda: set())
        for tile in self.tiles:
            for pattern in tile.patterns:
                lookup[pattern].add(tile)

        # These are the list of tiles which have at least one edge which is unsatisfied by another tile.
        edge_tiles = set(next(iter(tiles)) for pattern, tiles in lookup.items() if len(tiles) == 1)
        
        edge_counter = Counter(tile for tiles in lookup.values() if len(tiles) == 1 for tile in tiles)
        self.corner_tiles = set((tile for tile, matched_edge_count in edge_counter.items() if matched_edge_count == 4))
        assert len(self.corner_tiles) == 4

        for pattern, tiles in lookup.items():
            # We're assuming that there is only ever a single matching pair, so let's make sure that's the case
            assert len(tiles) <= 2

        for start_tile in self.corner_tiles:
            self.grid = [
                [None for j in range(self.width)]
                for i in range(self.height)
            ]

            self.grid[0][0] = self.make_edge(start_tile, lookup, "top", "left")
            
            outstanding_tiles = set(self.tiles) - set([self.grid[0][0]])

            valid_start_tile = True
            while valid_start_tile and outstanding_tiles: 
                for y in range(0, self.height):
                    for x in range(0, self.width):
                        if valid_start_tile and self.grid[y][x] is not None:
                            continue

                        found_match = False

                        for tile in outstanding_tiles:
                            required = {
                                
                            }

                            if tile in edge_tiles and x not in [0, self.width - 1] and y not in [0, self.height - 1]:
                                # Edge blocks must appear on edges
                                continue

                            if tile in self.corner_tiles and (x not in [0, self.width - 1] or y not in [0, self.height - 1]):
                                # Corner blocks can only appear in corners
                                continue

                            if x == 1:
                                # We can flip the left hand tile on the x-axis safely
                                if self.grid[y][0].right not in tile.patterns and self.grid[y][0].left in tile.patterns:
                                    self.grid[y][0].flip("x")
                            
                            if y == 1:
                                # We can flip the top tile on the y-axis safely
                                if self.grid[0][x].bottom not in tile.patterns and self.grid[0][x].top in tile.patterns:
                                    self.grid[0][x].flip("y")

                            if x > 0 and self.grid[y][x - 1] is not None:
                                required["left"] = self.grid[y][x - 1].right
                            if x < self.width - 1 and self.grid[y][x + 1] is not None:
                                required["right"] = self.grid[y][x + 1].left
                            if y > 0 and self.grid[y - 1][x] is not None:
                                required["top"] = self.grid[y - 1][x].bottom
                            if y < self.height - 1 and self.grid[y + 1][x] is not None:
                                required["bottom"] = self.grid[y + 1][x].top


                            if any(pattern not in tile.patterns for pattern in required.values()):
                                # The tile must contain all of the required patterns
                                #print(f"## Tile {tile.id} does not match {x},{y} (missing {', '.join(f'{pos}:{pattern}' for pos, pattern in required.items() if pattern not in tile.patterns)})")
                                continue

                            self.grid[y][x] = self.align(tile, required)
                            outstanding_tiles.remove(self.grid[y][x])
                            found_match = True
                            break

                        if not found_match:
                            self.print()
                            valid_start_tile = False
                            break

            

    def align(self, tile: ImageTile, required: Dict[str, str]):
        for target_edge, pattern in required.items():
            current_edge = tile.get_pattern_side(pattern)
            tile.move(current_edge, target_edge)

        return tile


    def make_edge(self, tile: ImageTile, lookup: Dict[str, Set[ImageTile]], *edges: str):
        patterns = (pattern for pattern in tile.patterns if len(lookup[pattern]) == 1)

        for pattern, target_edge in zip(patterns, edges):
            current_edge = tile.get_pattern_side(pattern)
            tile.move(current_edge, target_edge)
            

        for pattern, target_edge in zip(patterns, edges):
            current_edge = tile.get_pattern_side(pattern)
            tile.move(current_edge, target_edge)

        return tile


    @property
    def corner_product(self) -> int:
        product = 1
        for tile in self.corner_tiles:
            product *= tile.id

        return product

    def print(self):
        for y in range(self.height):
            for x in range(self.width):
                if self.grid[y][x] is None:
                    print("      ", end="")
                else:
                    print(f"{self.grid[y][x].id}  ", end="")
            print("")

        print("")

        for y in range(self.height):
            for y2 in range(10):
                for x in range(self.width):
                    if self.grid[y][x] is None:
                        print("          ", end=" ")
                    else:
                        print(self.grid[y][x].pixels[y2], end=" ")
                print("")
            print("")

    def borderless(self) -> str:
        lines = []
        for y in range(self.height):
            for y2 in range(8):
                line = []
                for x in range(self.width):
                    if self.grid[y][x] is None:
                        line.append("        ")
                    else:
                        line.append(self.grid[y][x].borderless()[y2])
                lines.append("".join(line))

        return "\n".join(lines)

    def print_borderless(self):
        print(self.borderless())


example_image = Image(example_tiles)
print(f"Example Corner Product (part 1: {example_image.corner_product})")
example_image.print()
example_image.print_borderless()

real_image = Image(real_tiles)
print(f"Real Corner Product (part 1: {real_image.corner_product})")
real_image.print()
real_image.print_borderless()

...# #.##.##... .####.###. .#####.... 

#..#.#..#. .#...#.... .#.#....#. .#.....#.. .#.#.#.#.. .#..#...## ###.....## #####..#.. ..#.##...# #.##.##... .####.###. .#####.... 
##.......# ##.....### #.#.....#. .......##. ...#...... ..##....#. .....#.... ..#..#.... .#...#.... .####..... ..#...##.. ..#.#.#... 
......#.## #######..# #....##... .#..#....# ##...#...# ##..#...## ##.##.#### #.......#. ...#..#... ..#.....#. .#....#... ....##...# 
#.......## #.......#. ..#.#..... ..#.##...# ##..#....# ##..##..#. ....#.#... .#......#. .#.#..##.. .##..#...# ##..#.#..# #.#..##... 
##.#..##.# ###.....## ##.....#.. .#.#....## #.##.#.... .......... ...#.....# #...##.... ......#.#. .#...#..## #....##.## #......#.. 
...#.#.... ..#..#.... ..#.#....# ##.#..#... ..#...##.# #...#..... .....#.... ......##.# #......### #.##.#...# ##.#..##.. ..##....## 
#.#..##... ..#......# #.....#.## #.##...... .#...#...# #...#..#.# #....#.#.# #.#...#..# #........# #..#...#.. ..##.##.## #......#.# 
##..#...#. .#...#...# #.#....

In [171]:
class PatternMatcher(object):
    def __init__(self, pattern: str):
        self.pattern = pattern

        pattern_lines = self.pattern.splitlines()
        self.height = len(pattern_lines)
        self.width = max(
            len(line) for line in pattern_lines
        )

    def find_all(self, source: str) -> Iterator[str]:
        source_lines = source.splitlines()

        for y in range(len(source_lines) - self.height):
            for x in range(len(source_lines[y]) - self.width):
                monster_candidate = list(line[x:x+self.width] for line in source_lines[y:y+self.height])
                if self.matches(monster_candidate):
                    yield "\n".join(monster_candidate)
        
    def matches(self, source: str) -> bool:
        if not isinstance(source, list):
            source = source.splitlines()

        for pline, sline in zip(self.pattern.splitlines(), source):
            for pchar, schar in zip(pline, sline):
                if pchar == '#' and schar != '#':
                    return False

        return True

class RotatedPatternMatcher(object):
    def __init__(self, pattern: str):
        lines = pattern.splitlines()
        rotated = "\n".join(
            "".join(lines[y][x] for y in range(len(lines)))
            for x in range(len(lines[0]))
        )

        self.patterns = [
            PatternMatcher(pattern),
            PatternMatcher(RotatedPatternMatcher.reverse(pattern)),
            PatternMatcher(RotatedPatternMatcher.rotate(pattern)),
            PatternMatcher(RotatedPatternMatcher.reverse(RotatedPatternMatcher.rotate(pattern))),
            PatternMatcher(RotatedPatternMatcher.rotate(RotatedPatternMatcher.reverse(pattern))),
            PatternMatcher(RotatedPatternMatcher.reverse(RotatedPatternMatcher.rotate(RotatedPatternMatcher.reverse(pattern))))
        ]

    def find_all(self, source: str) -> Iterator[str]:
        return set(
            match for pattern in self.patterns for match in pattern.find_all(source)
        )

    @staticmethod
    def reverse(pattern: str) -> str:
        return "\n".join(
            "".join(reversed(line))
            for line in pattern.splitlines()
        )

    @staticmethod
    def rotate(pattern: str) -> str:
        lines = pattern.splitlines()
        return "\n".join(
            "".join(lines[y][x] for y in range(len(lines)))
            for x in range(len(lines[0]))
        )

sea_monster = """
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
"""[1:-1]

sea_monster_roughness = sum(1 if pixel == '#' else 0 for pixel in sea_monster)
sea_monster = PatternMatcher(sea_monster)

assert sea_monster.matches("""
.#...#.###...#.##.O#
O.##.OO#.#.OO.##.OOO
#O.#O#.O##O..O.#O##.
"""[1:-1].replace('O', '#'))


sea_monster = RotatedPatternMatcher("""
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
"""[1:-1])

assert len(list(sea_monster.find_all(example_image.borderless()))) == 2


In [172]:
sea_monsters = len(sea_monster.find_all(real_image.borderless()))
total_roughness = sum(1 if pixel == '#' else 0 for pixel in real_image.borderless())
print(f"Environment Roughness (part 2): {total_roughness - (sea_monsters * sea_monster_roughness)}")

Environment Roughness (part 2): 2031
